In [2]:
import seaborn as sns
import csv
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
%matplotlib inline
sns.set()
import datetime


In [4]:

data_sample = pd.read_csv('DataSample__201709211530.csv')
data_sample['startDate'] = pd.to_datetime(data_sample['startDate'])

types = data_sample['type'].unique()
types

for t in types:
    f = lambda x: int(x.replace(',',''))
    # checks if the values contain decimal points, and uses a float conversion if so
    for v in data_sample.loc[data_sample['type'] == t, 'value'].head():
        if '.' in v:
            f = lambda x: float(x.replace(',',''))
            break
    data_sample.loc[data_sample['type'] == t, 'value'] = data_sample.loc[data_sample['type'] == t, 'value'].map(f)
for col in ['startDate', 'endDate', 'createdAt', 'updatedAt']:
    data_sample[col] = pd.to_datetime(data_sample[col], format='%Y-%m-%d %X')
to_tdelta = lambda row: row['endDate'] - row['startDate']
data_sample['duration'] = data_sample.apply(to_tdelta, axis=1)
data_sample['day_of_week'] = data_sample['startDate'].dt.dayofweek
data_sample.head() #0 - 6 is monday - sunday 

,_id,owner,source,sourceId,sourceName,type,startDate,endDate,value,content,originalData,createdAt,updatedAt,duration,day_of_week
0,1ff0fb89-6e1b-4601-84d0-98c79fdc5f2e,00000000-56ff-538b-2223-e1800b5e3ddb,healthkit:com.withings.wiScaleNG,A1DB680E-F9C4-4FA4-8818-1D35E398B5F0,Health Mate,health-bmi,2014-08-27 21:41:23,2014-08-27 21:41:23,26.7387,NaN,NaN,2017-08-31 11:20:49,2017-08-31 11:20:49,0 days,2
1,78219621-1da4-4e3b-87bc-f4885718c63f,00000000-56ff-538b-2223-e1800b5e3ddb,healthkit:com.withings.wiScaleNG,2030515A-1C70-40F3-8470-BDE99ABFC906,Health Mate,health-weight,2014-08-27 21:41:23,2014-08-27 21:41:23,71.042,NaN,NaN,2017-08-31 11:20:58,2017-08-31 11:20:58,0 days,2
2,2d917e7e-0844-4f52-b15e-2a5346c96dda,00000000-56ff-538b-2223-e1800b5e3ddb,healthkit:com.withings.wiScaleNG,96D21A44-67D7-412D-BF3A-543819A8EBF0,Health Mate,health-bmi,2014-08-28 06:24:53,2014-08-28 06:24:53,26.5866,NaN,NaN,2017-08-31 11:20:49,2017-08-31 11:20:49,0 days,3
3,ab800f4b-bef7-434e-8e57-e0fccc44bfb7,00000000-56ff-538b-2223-e1800b5e3ddb,healthkit:com.withings.wiScaleNG,5D73FD9D-17E1-480C-9235-3E5036503E6A,Health Mate,health-body-fat,2014-08-28 06:24:53,2014-08-28 06:24:53,0.258699,NaN,NaN,2017-08-31 11:20:53,2017-08-31 11:20:53,0 days,3
4,fb2a0865-17c3-4ac0-a782-749cc7befe12,00000000-56ff-538b-2223-e1800b5e3ddb,healthkit:com.withings.wiScaleNG,5A9AB03C-30C9-43B9-9742-02E237A6F36D,Health Mate,health-weight,2014-08-28 06:24:53,2014-08-28 06:24:53,70.638,NaN,NaN,2017-08-31 11:20:58,2017-08-31 11:20:58,0 days,3


,_id,owner,source,sourceId,sourceName,type,startDate,endDate,value,content,originalData,createdAt,updatedAt,duration
0,1ff0fb89-6e1b-4601-84d0-98c79fdc5f2e,00000000-56ff-538b-2223-e1800b5e3ddb,healthkit:com.withings.wiScaleNG,A1DB680E-F9C4-4FA4-8818-1D35E398B5F0,Health Mate,health-bmi,2014-08-27 21:41:23,2014-08-27 21:41:23,26.7387,NaN,NaN,2017-08-31 11:20:49,2017-08-31 11:20:49,00:00:00
1,78219621-1da4-4e3b-87bc-f4885718c63f,00000000-56ff-538b-2223-e1800b5e3ddb,healthkit:com.withings.wiScaleNG,2030515A-1C70-40F3-8470-BDE99ABFC906,Health Mate,health-weight,2014-08-27 21:41:23,2014-08-27 21:41:23,71.042,NaN,NaN,2017-08-31 11:20:58,2017-08-31 11:20:58,00:00:00
2,2d917e7e-0844-4f52-b15e-2a5346c96dda,00000000-56ff-538b-2223-e1800b5e3ddb,healthkit:com.withings.wiScaleNG,96D21A44-67D7-412D-BF3A-543819A8EBF0,Health Mate,health-bmi,2014-08-28 06:24:53,2014-08-28 06:24:53,26.5866,NaN,NaN,2017-08-31 11:20:49,2017-08-31 11:20:49,00:00:00
3,ab800f4b-bef7-434e-8e57-e0fccc44bfb7,00000000-56ff-538b-2223-e1800b5e3ddb,healthkit:com.withings.wiScaleNG,5D73FD9D-17E1-480C-9235-3E5036503E6A,Health Mate,health-body-fat,2014-08-28 06:24:53,2014-08-28 06:24:53,0.258699,NaN,NaN,2017-08-31 11:20:53,2017-08-31 11:20:53,00:00:00
4,fb2a0865-17c3-4ac0-a782-749cc7befe12,00000000-56ff-538b-2223-e1800b5e3ddb,healthkit:com.withings.wiScaleNG,5A9AB03C-30C9-43B9-9742-02E237A6F36D,Health Mate,health-weight,2014-08-28 06:24:53,2014-08-28 06:24:53,70.638,NaN,NaN,2017-08-31 11:20:58,2017-08-31 11:20:58,00:00:00
5,b80745d5-7f44-487f-a387-a1eb98cc42eb,00000000-56ff-538b-2223-e1800b5e3ddb,healthkit:com.withings.wiScaleNG,E4977A42-C4D6-4AC7-91A3-A11B4C5D4A83,Health Mate,health-body-fat,2014-08-28 23:46:27,2014-08-28 23:46:27,0.249602,NaN,NaN,2017-08-31 11:20:53,2017-08-31 11:20:53,00:00:00
6,049c2713-0bf8-44c4-8981-81866380667a,00000000-56ff-538b-2223-e1800b5e3ddb,healthkit:com.withings.wiScaleNG,B278E7C3-4509-4E1B-8C4C-EE469FBFCC15,Health Mate,health-weight,2014-08-28 23:46:27,2014-08-28 23:46:27,70.965,NaN,NaN,2017-08-31 11:20:58,2017-08-31 11:20:58,00:00:00
7,c229ed70-1325-4976-b1a5-141f885d1d7e,00000000-56ff-538b-2223-e1800b5e3ddb,healthkit:com.withings.wiScaleNG,544F96A4-915A-4844-8107-D15B369ABE19,Health Mate,health-bmi,2014-08-28 23:46:27,2014-08-28 23:46:27,26.7097,NaN,NaN,2017-08-31 11:20:49,2017-08-31 11:20:49,00:00:00
8,07b9fcbd-ddcb-44ee-9a80-4c628198cc46,00000000-56ff-538b-2223-e1800b5e3ddb,healthkit:com.withings.wiScaleNG,D887B5EA-014B-4D49-B315-EF1B22030B9A,Health Mate,health-body-fat,2014-08-29 06:45:05,2014-08-29 06:45:05,0.254801,NaN,NaN,2017-08-31 11:20:53,2017-08-31 11:20:53,00:00:00
9,2b245141-0f9d-42e1-a124-7f1abc56fc64,00000000-56ff-538b-2223-e1800b5e3ddb,healthkit:com.withings.wiScaleNG,A6A406C9-6CAE-4493-8886-3244E19CABB4,Health Mate,health-bmi,2014-08-29 06:45:05,2014-08-29 06:45:05,26.5576,NaN,NaN,2017-08-31 11:20:49,2017-08-31 11:20:49,00:00:00


In [ ]:
#Sleep -Lucas 
    #How long are they sleeping per day, what day highest/lowest - Matt
    #Disturbances during night - Matt 
    #Compared to national average - Matt
    # Correlate amount of sleep with avg steps - Matt 

In [ ]:
#Steps - Sebastian/Matt
    #What day do you step the most? Least? 
    #What time of day do you step the most? 
    #What is the correlation between steps and sleep?
    #What is the correlation between avg steps and BMI/Weight/Heart Rate/Body Fat/ by months? 
    #On what days of the week is the user taking less steps than their average? Less than average of other users? 

In [ ]:
#Heart-Rate - John
    #Heart Rate over time. 
    #In the past 90 days, average heart rate rates
    #Heart rate max/min past 90 days 
    #Is your heart rate lower/higher than the national average? 

In [ ]:
#Bmi/Body Fat/Weight -Luqmaan/Aydin 
    #How has BMI 
    #BMI average versus others 
    #BMI rate of change compared to other people's rates of change 
    #Weight over time/lowest weight/highest weight 
    #biggest weight change in a month span 
    #Weight change correlated with avg steps. Does avg steps imply more weight loss?